In [23]:
import pylab as plt
import pymoc
import xidplus
import numpy as np
%matplotlib inline
from astropy.table import Table, join

In [2]:
import seaborn as sns

This notebook uses all the raw data from the CIGALE predictions and photoz catalogue, maps, PSF and relevant MOCs to create XID+ prior object and relevant tiling scheme

## Read in MOCs
The selection functions required are the main MOC associated with the masterlist. 

In [13]:
Sel_func=pymoc.MOC()
Sel_func.read('../../dmu4/dmu4_sm_GAMA-15/data/holes_GAMA-15_ukidss_k_O16_20180327.fits')

## Read in CIGALE predictions catalogue

In [14]:
cigale=Table.read('../GAMA15_Ldust_prediction_results.fits')

In [15]:
cigale['id'].name = 'help_id'

## Read in photoz

In [16]:
photoz=Table.read('../../dmu24/dmu24_GAMA-15/data/master_catalogue_gama-15_20180119_photoz_20180210_r_optimised.fits')

In [17]:
photoz

help_id,RA,DEC,id,z1_median,z1_min,z1_max,z1_area,z2_median,z2_min,z2_max,z2_area,za_hb,chi_r_eazy,chi_r_atlas,chi_r_cosmos,chi_r_stellar,stellar_type
str27,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str6
HELP_J141546.547+000529.927,213.943944434,0.0916463485464,322065,0.9028,0.583,1.2407,0.78,1.6682,1.642,1.698,0.019,0.888999651538,0.0038901725,0.036354,0.070896275,1.8363205,m4iii
HELP_J144113.844-000720.633,220.307684538,-0.122398155882,322066,1.1282,0.5225,1.856,0.775,0.3912,0.3587,0.4212,0.021,0.860917897266,0.067539125,0.034600075,0.01283038,1.328487,f8v
HELP_J143631.813+002833.098,219.132553285,0.475860557929,322067,1.29,0.7952,1.8991,0.796,-99.0,-99.0,-99.0,-99.0,1.04812787974,0.0010802005,0.01882907,0.0368701,9.1299375,m1iii
HELP_J143302.417+002405.826,218.260071119,0.401618282766,322068,1.1347,0.3875,1.9517,0.798,-99.0,-99.0,-99.0,-99.0,1.00562871194,-99.0,-99.0,-99.0,-99.0,
HELP_J144038.334-002408.434,220.159726887,-0.402342819039,322069,1.0148,0.6858,1.4809,0.775,1.7087,1.6659,1.7552,0.024,0.866500650958,0.000401907,0.125252075,0.29048875,12.5883675,wg8iii
HELP_J143418.946-004636.350,218.578941517,-0.776763821989,322070,0.886,0.5089,1.2677,0.702,1.7168,1.5872,1.856,0.073,0.88334960273,0.2708705,0.57580525,0.44484775,6.14024,wk2iii
HELP_J143159.128-000214.789,217.996368068,-0.0374412653439,322071,0.8571,0.3958,1.2813,0.641,1.6539,1.3862,1.9165,0.156,0.88334960273,0.041801725,0.573121,0.220772575,2.48693275,wk2iii
HELP_J144020.768-002750.029,220.08653192,-0.463896849488,322072,0.9088,0.6908,1.2207,0.796,-99.0,-99.0,-99.0,-99.0,0.872100152911,0.11123585,0.70335325,0.6111925,18.1824775,wk2iii
HELP_J143048.452+010925.488,217.701882934,1.15707996473,322073,0.9517,0.6262,1.2677,0.796,-99.0,-99.0,-99.0,-99.0,0.888999651538,0.1052213,0.85642575,0.41004575,17.4037275,wk2iii


## Join CIGALE and photoz tables

In [18]:
prior=join(cigale,photoz)

In [19]:
from astropy.cosmology import Planck15 as cosmo
from astropy import units as u
f_pred=prior['bayes.dust.luminosity']/(4*np.pi*cosmo.luminosity_distance(prior['z1_median']).to(u.cm))

In [36]:
prior=prior[np.isfinite(f_pred.value)][np.log10(f_pred.value[np.isfinite(f_pred.value)])>8.5]

In [50]:
prior['DEC'].name='Dec'

## Read in Maps

In [43]:
pswfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/GAMA-15_SPIRE250_v0.9.fits'#SPIRE 250 map
pmwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/GAMA-15_SPIRE350_v0.9.fits'#SPIRE 350 map
plwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/GAMA-15_SPIRE500_v0.9.fits'#SPIRE 500 map

#output folder
output_folder='./'

In [47]:
from astropy.io import fits
from astropy import wcs

#-----250-------------
hdulist = fits.open(pswfits)
im250phdu=hdulist[0].header
im250hdu=hdulist[1].header

im250=hdulist[1].data*1.0E3 #convert to mJy
nim250=hdulist[3].data*1.0E3 #convert to mJy
w_250 = wcs.WCS(hdulist[1].header)
pixsize250=3600.0*w_250.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()
#-----350-------------
hdulist = fits.open(pmwfits)
im350phdu=hdulist[0].header
im350hdu=hdulist[1].header

im350=hdulist[1].data*1.0E3 #convert to mJy
nim350=hdulist[3].data*1.0E3 #convert to mJy
w_350 = wcs.WCS(hdulist[1].header)
pixsize350=3600.0*w_350.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()
#-----500-------------
hdulist = fits.open(plwfits)
im500phdu=hdulist[0].header
im500hdu=hdulist[1].header
im500=hdulist[1].data*1.0E3 #convert to mJy
nim500=hdulist[3].data*1.0E3 #convert to mJy
w_500 = wcs.WCS(hdulist[1].header)
pixsize500=3600.0*w_500.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()

In [48]:
## Set XID+ prior class

In [51]:
#---prior250--------
prior250=xidplus.prior(im250,nim250,im250phdu,im250hdu, moc=Sel_func)#Initialise with map, uncertianty map, wcs info and primary header
prior250.prior_cat(prior['RA'] ,prior['Dec'] ,'GAMA15_Ldust_prediction_results.fits',ID=prior['help_id'] )#Set input catalogue
prior250.prior_bkg(-5.0,5)#Set prior on background (assumes Gaussian pdf with mu and sigma)
#---prior350--------
prior350=xidplus.prior(im350,nim350,im350phdu,im350hdu, moc=Sel_func)
prior350.prior_cat(prior['RA'] ,prior['Dec'] ,'GAMA15_Ldust_prediction_results.fits',ID=prior['help_id'] )
prior350.prior_bkg(-5.0,5)

#---prior500--------
prior500=xidplus.prior(im500,nim500,im500phdu,im500hdu, moc=Sel_func)
prior500.prior_cat(prior['RA'] ,prior['Dec'] ,'GAMA15_Ldust_prediction_results.fits',ID=prior['help_id'] )
prior500.prior_bkg(-5.0,5)

In [55]:
#pixsize array (size of pixels in arcseconds)
pixsize=np.array([pixsize250,pixsize350,pixsize500])
#point response function for the three bands
prfsize=np.array([18.15,25.15,36.3])
#use Gaussian2DKernel to create prf (requires stddev rather than fwhm hence pfwhm/2.355)
from astropy.convolution import Gaussian2DKernel

##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0,1] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1,1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2,1] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires psf as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)

In [56]:
import pickle
#from moc, get healpix pixels at a given order
from xidplus import moc_routines
order=9
tiles=moc_routines.get_HEALPix_pixels(order,prior250.sra,prior250.sdec,unique=True)
order_large=6
tiles_large=moc_routines.get_HEALPix_pixels(order_large,prior250.sra,prior250.sdec,unique=True)
print('----- There are '+str(len(tiles))+' tiles required for input catalogue and '+str(len(tiles_large))+' large tiles')
output_folder='./'
outfile=output_folder+'Master_prior.pkl'
xidplus.io.pickle_dump({'priors':[prior250,prior350,prior500],'tiles':tiles,'order':order,'version':xidplus.io.git_version()},outfile)
outfile=output_folder+'Tiles.pkl'
with open(outfile, 'wb') as f:
    pickle.dump({'tiles':tiles,'order':order,'tiles_large':tiles_large,'order_large':order_large,'version':xidplus.io.git_version()},f)
raise SystemExit()

----- There are 4940 tiles required for input catalogue and 101 large tiles
writing total_bytes=1549524018...
writing bytes [0, 1073741824)... done.
writing bytes [1073741824, 1549524018)... done.


SystemExit: 

In [57]:
prior250.nsrc

1236678